In [31]:
import sklearn
sklearn.__version__

'1.7.2'

In [32]:
import pandas as pd

df = pd.read_csv("hiring_decisions.csv")
print(f"Instance number: {df.shape[0]}")
df.isnull().sum()

Instance number: 1500


id                     0
Age                    0
Gender                 0
EducationLevel         0
ExperienceYears        0
PreviousCompanies      0
DistanceFromCompany    0
InterviewScore         0
SkillScore             0
PersonalityScore       0
RecruitmentStrategy    0
HiringDecision         0
dtype: int64

In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, confusion_matrix 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [34]:
Y = df['HiringDecision']
X = df.drop(['id', 'HiringDecision'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/4, stratify=Y, random_state=0)

coltran = ColumnTransformer(transformers=[("std", StandardScaler(), ['Age', 'EducationLevel', 'ExperienceYears', 'PreviousCompanies', 'DistanceFromCompany', 'InterviewScore', 'SkillScore', 'PersonalityScore', 'RecruitmentStrategy'])], remainder='passthrough')
coltran.fit(X_train)
X_train = coltran.transform(X_train)
X_test = coltran.transform(X_test)

transformed_features = coltran.named_transformers_['std'].get_feature_names_out()
output_columns = list(transformed_features) + ['Gender']

X_train = pd.DataFrame(X_train, columns=output_columns)
X_test = pd.DataFrame(X_test, columns=output_columns)

In [35]:
model = SGDClassifier(random_state=121)
model.fit(X_train, Y_train)

trainpred = model.predict(X_train)
testpred = model.predict(X_test)

f1_train = f1_score(Y_train, trainpred)
f1_test = f1_score(Y_test, testpred)
confusion_train = confusion_matrix(Y_train, trainpred)
confusion_test = confusion_matrix(Y_test, testpred)

print(f"F1 Score train: {f1_train}")
print(f"F1 Score test: {f1_test}")
print(f"Confusion matrix on train: {confusion_train}")
print(f"Confusion matrix on test: {confusion_test}")

F1 Score train: 0.6012861736334405
F1 Score test: 0.6210045662100456
Confusion matrix on train: [[690  86]
 [162 187]]
Confusion matrix on test: [[224  35]
 [ 48  68]]


In [36]:
Y = df['HiringDecision']
X = df.drop(['id', 'HiringDecision'], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/4, stratify=Y, random_state=0)

coltran = ColumnTransformer(transformers=[("std", StandardScaler(), ['Age', 'EducationLevel', 'ExperienceYears', 'PreviousCompanies', 'DistanceFromCompany', 'InterviewScore', 'SkillScore', 'PersonalityScore', 'RecruitmentStrategy'])], remainder='passthrough')
coltran.fit(X_train)
X_train = coltran.transform(X_train)
X_test = coltran.transform(X_test)

model = SGDClassifier(random_state=121)
model.fit(X_train, Y_train)
res = model.predict(X_test)
X_test['pred'] = res

man = X_test[X_test['Gender'] == 0]
woman = X_test[X_test['Gender'] == 1]

prob_man = man[man['pred'] == 0].shape[0] / man.shape[0]
prob_woman = woman[woman['pred'] == 0].shape[0] / woman.shape[0]

print(f"Probability to hire a man: {1 - prob_man}")
print(f"Probability to hire a woman: {1 - prob_woman}")

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices